In [42]:
"""
Some sources: 

- https://github.com/wobine/blackboard101/blob/master/EllipticCurvesPart4-PrivateKeyToPublicKey.py
- https://wiki.openssl.org/index.php/Elliptic_Curve_Cryptography
- https://onyb.gitbook.io/secp256k1-python/point-addition-in-python
- https://github.com/vbuterin/pybitcointools

Most of the code below copied from https://onyb.gitbook.io/secp256k1-python/point-addition-in-python, check it out!

"""
from dataclasses import dataclass
import hashlib
from ripemd import RIPEMD160
from typing import Tuple, Union

@dataclass
class PrimeGaloisField:
    prime: int

    def __contains__(self, field_value: "FieldElement") -> bool:
        # called whenever you do: <FieldElement> in <PrimeGaloisField>
        return 0 <= field_value.value < self.prime
    
@dataclass
class FieldElement:
    value: int
    field: PrimeGaloisField

    def __repr__(self):
        return "0x" + f"{self.value:x}".zfill(64)
        
    @property
    def P(self) -> int:
        return self.field.prime
    
    def __add__(self, other: "FieldElement") -> "FieldElement":
        return FieldElement(
            value=(self.value + other.value) % self.P,
            field=self.field
        )
    
    def __sub__(self, other: "FieldElement") -> "FieldElement":
        return FieldElement(
            value=(self.value - other.value) % self.P,
            field=self.field
        )

    def __rmul__(self, scalar: int) -> "FieldValue":
        return FieldElement(
            value=(self.value * scalar) % self.P,
            field=self.field
        )

    def __mul__(self, other: "FieldElement") -> "FieldElement":
        return FieldElement(
            value=(self.value * other.value) % self.P,
            field=self.field
        )
        
    def __pow__(self, exponent: int) -> "FieldElement":
        return FieldElement(
            value=pow(self.value, exponent, self.P),
            field=self.field
        )

    def __truediv__(self, other: "FieldElement") -> "FieldElement":
        other_inv = other ** -1
        return self * other_inv


@dataclass
class EllipticCurve:
    a: int
    b: int

    field: PrimeGaloisField
    
    def __contains__(self, point: "Point") -> bool:
        x, y = point.x, point.y
        return y ** 2 == x ** 3 + self.a * x + self.b

    def __post_init__(self):
        # Encapsulate int parameters in FieldElement
        self.a = FieldElement(self.a, self.field)
        self.b = FieldElement(self.b, self.field)
    
        # Check for membership of curve parameters in the field.
        if self.a not in self.field or self.b not in self.field:
            raise ValueError

P: int = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
field = PrimeGaloisField(prime=P)

# Elliptic curve parameters A and B of the curve : y² = x³ Ax + B
A: int = 0
B: int = 7

secp256k1 = EllipticCurve(
    a=A,
    b=B,
    field=field
)


inf = float("inf")

@dataclass
class Point:
    x: int
    y: int

    curve: EllipticCurve

    def __post_init__(self):
        # Ignore validation for I
        if self.x is None and self.y is None:
            return

        # Encapsulate int coordinates in FieldElement
        self.x = FieldElement(self.x, self.curve.field)
        self.y = FieldElement(self.y, self.curve.field)

        # Verify if the point satisfies the curve equation
        if self not in self.curve:
            raise ValueError

    def __add__(self, other):
        if self == I:
            return other

        if other == I:
            return self

        if self.x == other.x and self.y == (-1 * other.y):
            return I

        if self.x != other.x:
            x1, x2 = self.x, other.x
            y1, y2 = self.y, other.y

            s = (y2 - y1) / (x2 - x1)
            x3 = s ** 2 - x1 - x2
            y3 = s * (x1 - x3) - y1

            return self.__class__(
                x=x3.value,
                y=y3.value,
                curve=secp256k1
            )

        if self == other and self.y == inf:
            return I

        if self == other:
            x1, y1, a = self.x, self.y, self.curve.a

            s = (3 * x1 ** 2 + a) / (2 * y1)
            x3 = s ** 2 - 2 * x1
            y3 = s * (x1 - x3) - y1

            return self.__class__(
                x=x3.value,
                y=y3.value,
                curve=secp256k1
            )
    
    def __rmul__(self, scalar: int) -> "Point":
        current = self
        result = I
        while scalar:
            if scalar & 1:  # same as scalar % 2
                result = result + current
            current = current + current  # point doubling
            scalar >>= 1  # same as scalar / 2
        return result


G = Point(
    x=0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
    y=0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8,
    curve=secp256k1
)

# Order of the group generated by G, such that nG = I
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

I = Point(x=None, y=None, curve=secp256k1)

def private_key_to_public_key(private_key: int) -> Tuple[int, int]:
    K = private_key*G
    return K.x.value, K.y.value

def sha(a: Union[str, bytes]) -> str:
    if isinstance(a, str):
        return hashlib.sha256(a.encode('utf-8')).hexdigest()
    return hashlib.sha256(a).hexdigest()

def ripemd160(a: str) -> str:
    return hashlib.new('ripemd160', a.encode('utf-8')).hexdigest()

def ripemd160_bytes(raw_adress: bytes) -> bytes:
    return RIPEMD160(sha(raw_adress)).digest()

code_strings = {
        2: '01',
        10: '0123456789',
        16: '0123456789abcdef',
        32: 'abcdefghijklmnopqrstuvwxyz234567',
        58: '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz',
        256: ''.join([chr(x) for x in range(256)])
    }
def get_code_string(base):
    if base in code_strings:
        return code_strings[base]
    else:
        raise ValueError("Invalid base!")
def encode(val, base, minlen=0):
    base, minlen = int(base), int(minlen)
    code_string = get_code_string(base)
    result_bytes = bytes()
    while val > 0:
        curcode = code_string[val % base]
        result_bytes = bytes([ord(curcode)]) + result_bytes
        val //= base

    pad_size = minlen - len(result_bytes)

    padding_element = b'\x00' if base == 256 else b'1' \
        if base == 58 else b'0'
    if (pad_size > 0):
        result_bytes = padding_element*pad_size + result_bytes

    result_string = ''.join([chr(y) for y in result_bytes])
    result = result_bytes if base == 256 else result_string

    return result

def raw_adress_from_public_key(public_key: Tuple[int]) -> bytes:
    return b'\x04' + encode(public_key[0], 256, 32) + encode(public_key[1], 256, 32)
    
def public_key_to_addres(public_key):
    return hashlib.sha256(public_key).hexdigest()

def base_58(num: int):
    """ Returns num in a base58-encoded string """
    encode = ''
    alphabet = '123456789abcdefghijkmnopqrstuvwxyzABCDEFGHJKLMNPQRSTUVWXYZ'
    base_count = len(alphabet)

    num = int(num, 16)
    if (num < 0):
        return ''

    while (num >= base_count):	
        mod = int(num % base_count)
        encode = alphabet[mod] + encode
        num = num // base_count

    if (num):
        encode = alphabet[num] + encode

    return encode

def base_58_check(raw_adress: str) -> str:
    prefix = '00'
    a = prefix + raw_adress
    checksum = sha(sha(a))[-4:]
    a += checksum
    return base_58(a)

In [47]:
private_key = hashlib.sha256(bytes('some big long brainwallet password', 'utf-8'))
priv = private_key.hexdigest()
priv
public_key = private_key_to_public_key(int(private_key.hexdigest(), 16))
# encode(public_key[0], 16)
# private_key_to_public_key(int(private_key.hexdigest(), 16)) == fast_multiply(G, decode(priv, 16))
x, y = encode(public_key[0], 16 , 64), encode(public_key[1], 16 , 64)
# f"04{x}{y}" == privtopub(priv)

'0420f34c2786b4bae593e22596631b025f3ff46e200fc1d4b52ef49bbdc2ed00b26c584b7e32523fb01be2294a1f8a5eb0cf71a203cc034ced46ea92a8df16c6e9'

In [45]:
encode(public_key[0], 16)

'20f34c2786b4bae593e22596631b025f3ff46e200fc1d4b52ef49bbdc2ed00b2'

In [129]:
private_key = '0x1E99423A4ED27608A15A2616A2B0E9E52CED330AC530EDCC32C8FFC6A526AEDD'
print(f"private key (hex): {private_key}")
public_key = private_key_to_public_key(int(private_key, 16))
print(f"public key x (hex) {hex(public_key[0])}")
print(f"public key y (hex) {hex(public_key[1])}")
raw_adress = raw_adress_from_public_key(public_key)
raw_adress_int = '0x04' + hex(public_key[0])[2:] + hex(public_key[1])[2:]
print(f"Raw Bitcoin address (bytes): {raw_adress}")
print(f"sha(raw bitcoin address): {sha(raw_adress)}")
payload = ripemd160_bytes(sha(raw_adress))
print(f"ripemd160(sha(raw bitcoin address)) {payload}")

private key (hex): 0x1E99423A4ED27608A15A2616A2B0E9E52CED330AC530EDCC32C8FFC6A526AEDD
public key x (hex) 0xf028892bad7ed57d2fb57bf33081d5cfcf6f9ed3d3d7f159c2e2fff579dc341a
public key y (hex) 0x7cf33da18bd734c600b96a72bbc4749d5141c90ec8ac328ae52ddfe2e505bdb
Raw Bitcoin address (bytes): b'\x04\xf0(\x89+\xad~\xd5}/\xb5{\xf30\x81\xd5\xcf\xcfo\x9e\xd3\xd3\xd7\xf1Y\xc2\xe2\xff\xf5y\xdc4\x1a\x07\xcf3\xda\x18\xbdsL`\x0b\x96\xa7+\xbcGI\xd5\x14\x1c\x90\xec\x8a\xc3(\xaeR\xdd\xfe.P[\xdb'
sha(raw bitcoin address): 8823c6b7765421882361f3ed3d1309a3a42ee81d2cdccbd5a68ba99198320fe0
ripemd160(sha(raw bitcoin address)) b'\x80b\x8b\xff\xdf\x03s\xbcB^N\xa7\x9d\n\x96Gs\x94\xae\x07'


In [135]:
print(ripemd160(sha(raw_adress_int)))
print(ripemd160_bytes(sha(raw_adress_int)))

4abc4aef1cb46d04c66bdf87931c9715fd55e6c0
b'\xcd"a{\x00r\xc2\xaf\x99\n\xcd\xe9+k",q\xf9\xce`'


In [23]:
# private_key = sha('something_random')
private_key = '0x038109007313a5807b2eccc082c8c3fbb988a973cacf1a7df9ce725c31b14776'
print(f"private key (hex): {private_key}")
_public_key = public_key(int(private_key, 16))
print(f"public key x (hex) {hex(_public_key[0])}")
print(f"public key y (hex) {hex(_public_key[1])}")

private key (hex): 0x038109007313a5807b2eccc082c8c3fbb988a973cacf1a7df9ce725c31b14776
public key x (hex) 0x2a406624211f2abbdc68da3df929f938c3399dd79fac1b51b0e4ad1d26a47aa
public key y (hex) 0x9f3bc9f3948a19dabb796a2a744aae50367ce38a3e6b60ae7d72159caeb0c102
